## Build Functions for ELT

In [114]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [115]:
import pandas as pd
import numpy as np
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
f.dl_tf_pd('NVDA','5m','1mo', skip_dl=False)

2024-10-05 04:24:56



,model,prediction,prob_no_change,prob_up,prob_down,kelly_criterion_1_2
0,XGBoost,0,0.750000,0.000000,0.250000,0.625000
1,RandomForest,0,0.951470,0.035975,0.012554,0.927205
2,GradientBoosting,0,0.779380,0.100786,0.119834,0.669070
3,KNN,0,0.944882,0.035390,0.019729,0.917323


In [111]:
symbol='NVDA'
interval='5m'
period='1mo'

# f.download(symbol, interval, period)
f.transform(symbol, interval, period)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)
f.predictions_summary(predictions, prediction_probas)

,model,prediction,prob_no_change,prob_up,prob_down,kelly_criterion_1_2
0,XGBoost,0,0.750000,0.000000,0.250000,0.625000
1,RandomForest,0,0.951470,0.035975,0.012554,0.927205
2,GradientBoosting,0,0.779380,0.100786,0.119834,0.669070
3,KNN,0,0.944882,0.035390,0.019729,0.917323


In [112]:
classification_reports

{'XGBClassifier': {'0': {'precision': 0.702928870292887,
   'recall': 0.7601809954751131,
   'f1-score': 0.7304347826086957,
   'support': 221.0},
  '1': {'precision': 0.8283828382838284,
   'recall': 0.8019169329073482,
   'f1-score': 0.814935064935065,
   'support': 313.0},
  '2': {'precision': 0.7731958762886598,
   'recall': 0.7525083612040134,
   'f1-score': 0.7627118644067796,
   'support': 299.0},
  'accuracy': 0.773109243697479,
  'macro avg': {'precision': 0.7681691949551251,
   'recall': 0.7715354298621583,
   'f1-score': 0.7693605706501799,
   'support': 833.0},
  'weighted avg': {'precision': 0.7752901269242205,
   'recall': 0.773109243697479,
   'f1-score': 0.7737714402626941,
   'support': 833.0}},
 'RandomForestClassifier': {'0': {'precision': 0.6583333333333333,
   'recall': 0.7149321266968326,
   'f1-score': 0.6854663774403471,
   'support': 221.0},
  '1': {'precision': 0.7936507936507936,
   'recall': 0.7987220447284346,
   'f1-score': 0.7961783439490446,
   'support'

In [121]:
# Pretty-print the dictionary
pprint.pprint(classification_reports, width=120)

{'GradientBoostingClassifier': {'0': {'f1-score': 0.7456896551724138,
                                      'precision': 0.7119341563786008,
                                      'recall': 0.7828054298642534,
                                      'support': 221.0},
                                '1': {'f1-score': 0.8112582781456954,
                                      'precision': 0.8419243986254296,
                                      'recall': 0.7827476038338658,
                                      'support': 313.0},
                                '2': {'f1-score': 0.7591973244147158,
                                      'precision': 0.7591973244147158,
                                      'recall': 0.7591973244147158,
                                      'support': 299.0},
                                'accuracy': 0.7743097238895558,
                                'macro avg': {'f1-score': 0.772048419244275,
                                              'precision': 0.

## Kelly Criterion

In [92]:
# fraction = p / l - q / g
# p = probability of success
# q = 1 - p = probability or failure
# l = % loss (ex: $10 -> $9 would mean l = .1)
# g = % gain (ex: $10 -> $12 would mean g = .2)

def kelly_c(p, l=1, g=2): # default l=1 and g=2 for 2 to 1 gain to loss
    return p / l - (1 - p) / g

In [93]:
kelly_c(.75,1,2)

0.625

In [94]:
kelly_c(.75)

0.625

## Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')